# Una visión general de `astropy.coordinates`:


### *Nota: Este notebook es una copia del notebook del tutorial con algunas celdas redundantes omitidas y con las soluciones de los ejercicios incluidas*

### Importaciones preliminares

Comenzamos importando algunos paquetes generales que necesitaremos más adelante:

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

Nota: Si la siguiente celda falla, asegúrate de haber instalado `astroquery` ejecutando:
```
pip install astroquery
```
en tu terminal. Si es así, necesitarás ir a "Kernel->Restart" en este notebook después de la instalación.

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file

## Representando ubicaciones en el cielo con `astropy.coordinates`

En Astropy, la forma más común de representar y trabajar con coordenadas celestes es usar el objeto `SkyCoord`. Un `SkyCoord` puede crearse directamente a partir de ángulos o arreglos de ángulos con sus unidades asociadas, como se muestra a continuación.

Para comenzar, supongamos que queremos crear un objeto `SkyCoord` para el centro del cúmulo abierto NGC 188, de modo que más adelante podamos consultar y obtener las estrellas que podrían ser miembros del cúmulo. Supongamos también, por ahora, que ya conocemos las coordenadas celestes del cúmulo como (12.11, 85.26) grados en el sistema de coordenadas ICRS. El ICRS — a veces referido simplemente como coordenadas "ecuatoriales" o "J2000" — es actualmente el sistema de coordenadas astronómicas más común para la astronomía estelar o extragaláctica, y es el sistema de coordenadas predeterminado para `SkyCoord`. Por lo tanto, podríamos crear un objeto `SkyCoord` para el centro de NGC 188 así:

In [ ]:
ngc188_center = SkyCoord(12.11*u.deg, 85.26*u.deg)
ngc188_center

Generalmente se recomienda especificar explícitamente el sistema de coordenadas (frame) en el que están tus coordenadas; en este caso, como ICRS es el valor predeterminado, esta sería una forma equivalente de crear nuestro objeto `SkyCoord`:

In [ ]:
ngc188_center = SkyCoord(12.11*u.deg, 85.26*u.deg, frame='icrs')
ngc188_center

Como veremos más adelante, existen muchos otros sistemas de coordenadas compatibles.

En las inicializaciones anteriores, pasamos objetos `astropy.units.Quantity` con unidades angulares para especificar los componentes angulares de nuestras coordenadas celestes. Sin embargo, `SkyCoord` también acepta coordenadas en formato de cadena (string), ya sea como cadenas separadas para RA/Dec o como una sola cadena:

In [ ]:
SkyCoord('00h48m26.4s', '85d15m36s', frame='icrs')

Algunas representaciones en formato de cadena no definen explícitamente las unidades, por lo que tendrías que especificarlas explícitamente si no están incluidas en la cadena:

In [ ]:
SkyCoord('00:48:26.4 85:15:36', unit=(u.hour, u.deg), 
         frame='icrs')

Para más información y ejemplos sobre cómo inicializar objetos `SkyCoord`, [consulta esta documentación](http://docs.astropy.org/en/latest/coordinates/skycoord.html).

El objeto `SkyCoord` que creamos tiene varias formas de acceder a la información contenida en él. Todos los objetos `SkyCoord` tienen atributos que te permiten obtener los datos de los componentes de las coordenadas, pero los nombres de estos componentes cambian según el sistema de coordenadas del `SkyCoord` que tengas. 

En nuestro caso, tenemos un `SkyCoord` con coordenadas ICRS: por lo tanto, los nombres de los componentes son abreviaturas en minúsculas de Ascensión Recta, `.ra`, y Declinación, `.dec`:

In [ ]:
ngc188_center.ra, ngc188_center.dec

Estos atributos `ra` y `dec` devuelven objetos especializados de tipo [``Quantity``](http://docs.astropy.org/en/stable/units/index.html) (en realidad, una subclase llamada [``Angle``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Angle.html), que a su vez es subclase de [``Latitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Latitude.html) y [``Longitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Longitude.html)).  

Estos objetos almacenan ángulos y proporcionan representaciones visualmente agradables de dichos ángulos, además de algunos atributos útiles para convertir rápidamente a unidades angulares comunes. Por ejemplo, en un notebook de Jupyter, estos objetos saben cómo representarse a sí mismos usando Latex:

In [ ]:
ngc188_center.ra

In [ ]:
ngc188_center.dec

También podemos obtener los valores de los componentes en diferentes unidades:

In [ ]:
ngc188_center.ra.hour, ngc188_center.ra.radian, ngc188_center.ra.degree

Y formatear los valores en cadenas con unidades especificadas ([documentación](http://docs.astropy.org/en/latest/coordinates/formatting.html)), por ejemplo:

In [ ]:
ngc188_center.ra.to_string(unit=u.hourangle, sep=':')

Ahora que tenemos un objeto `SkyCoord` para el centro de NGC 188, podemos seleccionar fuentes del catálogo *Gaia* Data Release 2 alrededor de esta posición para intentar encontrar estrellas que podrían ser miembros del cúmulo. Para hacer esto, usaremos el módulo `astroquery.gaia` para consultar el archivo de datos de *Gaia* ([documentación](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html)).

Esto requiere una conexión a internet, pero si falla, el archivo del catálogo está incluido en el repositorio para que puedas cargarlo localmente (es decir, puedes saltarte la siguiente celda si no tienes conexión a internet):

In [ ]:
from astroquery.gaia import Gaia

In [ ]:
table = QTable.read('gaia_results.fits')

In [ ]:
len(table)

Esta tabla ahora contiene aproximadamente 5000 estrellas del *Gaia* DR2 alrededor de la posición coordenada del centro de NGC 188. Ahora construyamos un objeto `SkyCoord` con la tabla de resultados. En el archivo de datos de *Gaia*, las coordenadas ICRS de una fuente están dadas en las columnas llamadas `"ra"` y `"dec"`:

In [ ]:
table['ra']

In [ ]:
table['dec']

Note que, dado que el archivo de datos de *Gaia* proporciona tablas con unidades asociadas, y que leímos esta tabla usando el objeto `QTable` ([docs](http://docs.astropy.org/en/latest/table/mixin_columns.html#quantity-and-qtable)), las columnas anteriores están representadas como objetos `Quantity` con unidades en grados. También observe que estas columnas contienen muchos valores de coordenadas (>5000): podemos pasarlos directamente a `SkyCoord` para obtener un único objeto `SkyCoord` que represente todas estas coordenadas:

In [ ]:
gaia_coords = SkyCoord(table['ra'], table['dec'])
gaia_coords

### Ejercicios

Crea un objeto `SkyCoord` del cúmulo abierto Las Pléyades (ya sea buscando las coordenadas y pasándolas directamente, o usando el método conveniente que aprendimos arriba):

In [ ]:
pleiades_center = SkyCoord.from_name('Pleiades')

Usando una única llamada a un método/función sobre el objeto `SkyCoord` que representa el centro de NGC 188, imprime una cadena con la RA/Dec en el formato 'HH:MM:SS.S DD:MM:SS.S'. Verifica tu respuesta con [SIMBAD](http://simbad.u-strasbg.fr/simbad/), que te mostrará las coordenadas sexagesimales del objeto.

<sub>(Pista: `SkyCoord.to_string()` puede ser útil)</sub>

In [ ]:
ngc188_center.to_string(style="hmsdms", sep=":", precision=1)

Usando una única llamada a un método/función sobre el objeto `SkyCoord` que contiene los resultados de nuestra consulta a *Gaia*, calcula la separación angular entre cada estrella resultante y las coordenadas del centro del cúmulo.

<sub>(Pista: `SkyCoord.separation()` puede ser útil)</sub>

In [ ]:
gaia_coords.separation(ngc188_center)

---

## Más que solo posiciones en el cielo: Incluyendo información de distancia en `SkyCoord`

Hasta ahora, solo hemos usado `SkyCoord` para representar posiciones en el cielo (es decir, solo `ra` y `dec`). A veces es útil incluir información de distancia junto con las coordenadas celestes, especificando así completamente la posición 3D de una fuente dada. Para pasar información de distancia, `SkyCoord` acepta un argumento clave llamado `distance`. Por lo tanto, si también conociéramos la distancia a NGC 188, podríamos pasar esa distancia (como un objeto `Quantity`) usando este argumento:

In [ ]:
ngc188_center_3d = SkyCoord(12.11*u.deg, 85.26*u.deg,
                            distance=1.96*u.kpc)

Con los resultados de nuestra consulta al catálogo *Gaia*, también se nos proporcionan mediciones de paralaje para cada estrella del catálogo. La forma predeterminada de pasar una distancia, como se mostró anteriormente, es pasando un objeto `Quantity` con una unidad de longitud. Sin embargo, `astropy.coordinates` también ofrece un objeto especializado llamado `Distance`, diseñado para manejar transformaciones comunes entre diferentes representaciones de distancia ([documentación](http://docs.astropy.org/en/latest/coordinates/index.html#distance)). 

Entre otras funcionalidades, esta clase permite pasar un valor de paralaje <sub>(pero tenga en cuenta que la transformación a distancia simplemente invierte el paralaje, ¡lo cual no siempre es recomendable!)</sub>.

In [ ]:
Distance(parallax=1*u.mas)

El catálogo de estrellas que consultamos en *Gaia* también contiene información de paralaje, por lo que podemos crear un objeto `SkyCoord` para representar las posiciones 3D de todas las estrellas de *Gaia*. Aquí, seleccionamos de la tabla solo aquellas estrellas con valores de paralaje positivos:

In [ ]:
table_3d = table[table['parallax'] > 0]

gaia_dist = Distance(parallax=table_3d['parallax'])
gaia_coords_3d = SkyCoord(table_3d['ra'], table_3d['dec'],
                          distance=gaia_dist)
gaia_coords_3d

Ahora que tenemos información de la posición 3D tanto para el centro del cúmulo como para las estrellas que consultamos en *Gaia*, podemos calcular la separación 3D (distancia) entre todas las fuentes de *Gaia* y el centro del cúmulo:

In [ ]:
sep3d = gaia_coords_3d.separation_3d(ngc188_center_3d)
sep3d

### Ejercicios

Usando los valores de separación 3D, define una máscara booleana para seleccionar candidatos a miembros del cúmulo. Selecciona todas las estrellas que estén dentro de 50 parsecs del centro del cúmulo. ¿Cuántos candidatos a miembros de NGC 188 tenemos, basándonos en sus posiciones 3D?

In [ ]:
ngc188_3d_mask = sep3d < 50*u.pc
ngc188_3d_mask.sum()